# Initial Package/Data Load

In [ ]:
import pandas as pd
import numpy as np
from timeit import time
import sklearn.metrics as metrics
from sklearn.utils import resample
from tqdm import tqdm
import gc
from datasets import Dataset


In [ ]:
google_colab = False

if google_colab == True:
    
    #for colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    df = pd.read_csv('/content/drive/MyDrive/dissertation_data/MLdata.csv')
else:
    
    df = pd.read_csv('/Users/josephmeyer/Desktop/git/NLP/dissertation/MLdata.csv')


In [ ]:
df.head()

# Basic Data Manipulation

In [ ]:
#concatenate responses
df['Response'] = df['open_ended_1'] + df['open_ended_2'] + df['open_ended_3'] + df['open_ended_4'] + df['open_ended_5']

In [ ]:
df.head()

In [ ]:
df[1080:1089]

# Exploratory Data Analysis

In [ ]:
#length
len(df)

In [ ]:
len(df.loc[df.Dataset == 'Dev'])

In [ ]:
len(df.loc[df.Dataset == 'Test'])

In [ ]:
df.dtypes

In [ ]:
#check average string length
res = [x for x in df['Response']]
length = [len(ele) for ele in res]
result = 0 if len(length) == 0 else (float(sum(length)) / len(length)) 
print("The Average length of String in list is : " + str(result))

In [ ]:
round(df[['E_Scale_score','A_Scale_score','O_Scale_score','C_Scale_score','N_Scale_score']].describe(), 2)

In [ ]:
descriptives = round(df[['E_Scale_score','A_Scale_score','O_Scale_score','C_Scale_score','N_Scale_score']].describe(), 2)

if google_colab == True:
    from google.colab import files
    descriptives.to_csv('descriptives.csv') 
    files.download('descriptives.csv')
else:
    descriptives.to_csv('/Users/josephmeyer/Desktop/git/NLP/dissertation/descriptives.csv')

print('Downloaded')


# Text Cleaning for Bag of Words

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

In [ ]:
#set stop words
stop_words = set(stopwords.words('english'))
#set lemmatizer
ps = PorterStemmer()

In [ ]:
def clean_text(text):
    """
    
    1. Lowercases text
    2. Removes punctuation
    3. Removes numbers
    4. Removes stop words
    5. Lemmatizes text
    
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = "".join([i for i in text if not i.isdigit()])
    text = " ".join([w for w in text.split() if w not in stop_words])
    text = " ".join([ps.stem(w) for w in text.split()])

    return text

In [ ]:
df['Cleaned_response'] = df['Response'].apply(clean_text)

In [ ]:
df.head()

In [ ]:
df['Cleaned_response'][0]

# Train/Test Data Creation

In [ ]:
#concatenate eval and train dataset
train = df.loc[df.Dataset == 'Train']
train.head()

In [ ]:
#Training variables

y_train = train['Response']
y_E_train = train['E_Scale_score']
y_A_train = train['A_Scale_score']
y_O_train = train['O_Scale_score']
y_C_train = train['C_Scale_score']
y_N_train = train['N_Scale_score']

y_train_list = [y_E_train, y_A_train, y_O_train, y_C_train, y_N_train]

In [ ]:
test = df.loc[df.Dataset == 'Test']
test.head()

In [ ]:
#test variables
x_test = test['Response']
y_E_test = test['E_Scale_score']
y_A_test = test['A_Scale_score']
y_O_test = test['O_Scale_score']
y_C_test = test['C_Scale_score']
y_N_test = test['N_Scale_score']

#create list of output variables
#e,a,o,c,n

y_test_list = [y_E_test, y_A_test, y_O_test, y_C_test, y_N_test]

# Bag of Words Feature Vector

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
#list comprehension to create list of docs
documents = [x for x in df['Cleaned_response']]

In [ ]:
documents[0]

In [ ]:
documents[0:2]

In [ ]:
#vectorizer, extract unigrams and bigrams
#max df remove words occuring in more than x% of documents
#min df remove words occuring less than x%
#x max features

# vectorizer = CountVectorizer(max_df = 0.4, ngram_range = (1,2), min_df = 0.02, max_features = 5000)

vectorizer = CountVectorizer(ngram_range = (1,1))

In [ ]:
#create vectors, #default is by unigram
bow = vectorizer.fit_transform(documents)

In [ ]:
#view features
print(vectorizer.get_feature_names()[0:10])

In [ ]:
bow.shape

In [ ]:
bow.toarray()

In [ ]:
#dictionary of words and indices
#vectorizer.vocabulary_

In [ ]:
#convert to dataframe
import scipy.sparse
from pandas import DataFrame

In [ ]:
bow_df = pd.DataFrame.sparse.from_spmatrix(bow)

In [ ]:
bow_df

In [ ]:
#calculate sparseness out of curiousity
round((bow_df.to_numpy() == 0).mean(), 4)

# Bag of Words train/test resplit

In [ ]:
#slice based on indicies
bow_x_train = bow_df[0:1088]

In [ ]:
bow_x_train.head()

In [ ]:
#verify length
print(len(bow_x_train))
print(len(bow_x_train) == len(train))

In [ ]:
bow_x_test = bow_df[1388:]

In [ ]:
print(len(bow_x_test))
print(len(bow_x_test) == len(test))

# Bag of Words Model Training/Testing

In [ ]:
# !pip install xgboost

In [ ]:
import sklearn
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
import sklearn.metrics as metrics
from scipy.stats import pearsonr
from sklearn.exceptions import ConvergenceWarning

In [ ]:
def regression_grid_train_test_report(model, x_train, y_train, x_test, y_test, paramater_grid, cv, score):
    
    
    """
    
    """
    
    #start timer
    start = time.time()

    
    print('\n Performing grid search....hold tight... \n =============================')
        
    model_name = model
    construct = y_test.name
    
    ###### grid search
    
    
    #construct grid search
    #number of parameter settings set to 60
    gs = RandomizedSearchCV(model, param_distributions = paramater_grid, scoring = score, cv = cv, n_iter = 60, random_state = 152) 
    
    
    #fit on training data
    gs.fit(x_train, y_train)
    best_parameters = gs.best_params_
    best_estimator = gs.best_estimator_
    
    
    print('Grid Search Complete')
    print('==================================')
    
    ##### predict on test data
    y_pred = best_estimator.predict(x_test) 
    
    
    ###### regression report
        
        
    print(f'Outcome Variable: {construct}')
    
    
    #number of grid search combinations
    
    
    n_iterations = 1
    
    for value in paramater_grid.values():
        n_iterations *= len(value)
    
    
    print(f'Number of original grid search combinations: {n_iterations}')
    
    
    print(f'Best parameters for {model_name} were {best_parameters}')
    
    
    print('\n Results Below')
    
    # mse=metrics.mean_squared_error(y_test, y_pred) 
    # print('MSE: ', round(mse,4))

    r = pearsonr(y_test, y_pred)
    print('r: ', r)
    
    print()
    print('==================================')
    
    #create global variable to access it out of function
    
    global frame
    
    
    #results data frame
    
    frame = pd.DataFrame([[construct, model_name, r[0]]],columns=['construct', 'model_name', 'r'])

   
    end = time.time()
    
    time_elapsed = (end - start) / 60
    
    print(f'Time Elapsed: {time_elapsed} minutes')
    
    
    print('\n \n \n Analysis Complete')
    
    
    
    return frame, r[0]
    
   
    

### Grid search

In [ ]:
#consider using randomized grid search

#configure models

rf = RandomForestRegressor(n_jobs = -1, verbose = 0)

enet = ElasticNet()

#parameter grids

enet_param_grid = { 
                    'alpha': np.linspace(0.1, 1, 20),
                    'l1_ratio': np.arange(0, 1.0, 0.05)}

#create list of parameter grids for iterations
# param_grids = [rf_param_grid, enet_param_grid]
models = [enet]
param_grids = [enet_param_grid]

In [ ]:
n_iterations = 1

for value in enet_param_grid.values():
    n_iterations *= len(value)
    
print(n_iterations)

In [ ]:
#ignore elastic net convergence warnings

from sklearn.exceptions import ConvergenceWarning

import warnings
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Modeling

In [ ]:
#regression_grid_train_test_report
#model, x_train, y_train, x_test, y_test, paramater_grid, cv, scoring

#run function across different models and parameter grids in my list, append to dfs


dfs = []
boot_results = []

for m,p in tqdm(zip(models, param_grids)):
  gc.collect()
  frame, r = regression_grid_train_test_report(m, bow_x_train, y_E_train, bow_x_test, y_E_test, p, 5, 'explained_variance')
  dfs.append(frame)
  boot_results.append(r)

E_bow_output = pd.concat(dfs)
E_bow_output

In [ ]:
E_bow_output = E_bow_output[['model_name', 'r']]

In [ ]:
E_bow_output.to_csv(path_or_buf = '/Users/josephmeyer/Desktop/git/NLP/dissertation/E_bow_output.csv')

# Rest of models

In [ ]:
pip install caffeine

In [ ]:
stimulant = True

if stimulant == True:
    import caffeine

In [ ]:
#create list of output variables
#e,a,o,c,n

y_train_list = [y_A_train, y_O_train, y_C_train, y_N_train]

y_test_list = [y_A_test, y_O_test, y_C_test, y_N_test]

In [ ]:
#create nested for loop that train all models and all outcomes
#two models
#two parameter grids
#five (four now) outcome variables

for i in range(0, len(y_train_list)):
    for x in range(0, len(models)):
        print(i,x, x)

In [ ]:
results = []
corrs = []

for i in tqdm(range(0, len(y_train_list))):
    for n in tqdm(range(n_iterations)):
        frame, r = regression_grid_train_test_report(enet, bow_x_train, y_train_list[i], bow_x_test, y_test_list[i], enet_param_grid, 5, 'explained_variance')
        results.append(frame)
        corrs.append(r)

bow_all_output = pd.concat(results)

bow_all_output.to_csv(path_or_buf = '/Users/josephmeyer/Desktop/git/NLP/dissertation/bow_aocn_output.csv')

# Empath

In [ ]:
!pip install empath

In [ ]:
stimulant = True

In [ ]:
if stimulant == True:
    import caffeine

In [ ]:
from empath import Empath
lexicon = Empath()

In [ ]:
empath_x_train_list = [x for x in train['Cleaned_response']]
empath_x_test_list = [x for x in test['Cleaned_response']]

In [ ]:
list_of_empath = []
for x in empath_x_train_list:
    empath = lexicon.analyze(x)
    list_of_empath.append(empath)

In [ ]:
empath_x_train = pd.DataFrame(list_of_empath)

In [ ]:
empath_x_train.head(10)

In [ ]:
len(empath_x_train)

In [ ]:
list_of_empath = []
for x in empath_x_test_list:
    empath = lexicon.analyze(x)
    list_of_empath.append(empath)
    
empath_x_test = pd.DataFrame(list_of_empath)
print(len(empath_x_test))

In [ ]:
empath_x_test.head(10)

# Fit Empath Algorithms

In [ ]:
#create list of output variables
#e,a,o,c,n

y_train_list = [y_E_train, y_A_train, y_O_train, y_C_train, y_N_train]

y_test_list = [y_E_test, y_A_test, y_O_test, y_C_test, y_N_test]

In [ ]:
#run function across different models and parameter grids in my list, append to dfs

results_empath = []
boot_empath = []

for i in tqdm(range(0, len(y_train_list))):
    for n in tqdm(range(n_iterations)):
        frame, r = regression_grid_train_test_report(enet, empath_x_train, y_train_list[i], empath_x_test, y_test_list[i], enet_param_grid, 5, 'explained_variance')
        results_empath.append(frame)
        boot_empath.append(r)

In [ ]:
empath_all_output = pd.concat(results_empath)

empath_all_output.to_csv(path_or_buf = '/Users/josephmeyer/Desktop/git/NLP/dissertation/output/empath_results.csv')

print('Results Saved')

# LSTM

In [ ]:
#!pip install gensim
%pip install keras

In [ ]:
import gensim.downloader as api
from keras.layers import Bidirectional
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Dropout
from keras.utils.vis_utils import plot_model
from keras.initializers import Constant
from keras.constraints import maxnorm
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras import regularizers
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
#clean text for lstm

import re
import nltk

def clean_text2(text):
    """
    1. Lowercases text
    2. Removes punctuation
    3. Removes numbers
    
    """
    text = text.lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = "".join([i for i in text if not i.isdigit()])

    return text

In [ ]:
df['lstm_text'] = df['Response'].apply(clean_text2)

In [ ]:
lstm_train = df.loc[df.Dataset == 'Train']
lstm_test = df.loc[df.Dataset == 'Test']

lstm_x_train = lstm_train['lstm_text']
lstm_x_test = lstm_test['lstm_text']

In [ ]:
#check vocabulary size
from collections import Counter
results = Counter()
df['lstm_text'].str.lower().str.split().apply(results.update)
print(f'length of results: {len(results)}')

#set vocabulary size and embedding size
voc_size = len(results)

#check for longest length for padding purposes
list = [x for x in df['lstm_text']]
longest = max(list, key = len)
max_length = len(longest)
print(f' max length is: {max_length}')

#unique responses
unique = set([x for x in df['lstm_text']])
print(f'length of unique: {len(unique)}')


#tokenize
tokenizer = Tokenizer(num_words=voc_size)
tokenizer.fit_on_texts(lstm_x_train)


#pad
sequences = tokenizer.texts_to_sequences(lstm_x_train.values)
lstm_x_train = pad_sequences(sequences,maxlen=max_length)

print(f'x_train shape is : {lstm_x_train.shape}')


#tokenize
tokenizer.fit_on_texts(lstm_x_test)


#pad
test_sequences = tokenizer.texts_to_sequences(lstm_x_test.values)
lstm_x_test = pad_sequences(test_sequences,maxlen=max_length)

print(f'x_test shape is : {lstm_x_test.shape}')

In [ ]:
#check indices
tokenizer.word_index

In [ ]:
print(lstm_x_train)

In [ ]:
print(lstm_x_test)

# Pre-trained Word Embeddings

In [ ]:
!python -m spacy download en_core_web_md
import en_core_web_md

In [ ]:
nlp = en_core_web_md.load()

In [ ]:
test = nlp('Hello World')
for token in test:
    print(len(token.vector))

In [ ]:
embedding_dim = 300

In [ ]:
embedding_matrix = np.zeros((voc_size, embedding_dim))

In [ ]:
#append word vectors in vocabulary to embedding matrix
for i, word in enumerate(tokenizer.word_index):
    embedding_matrix[i] = nlp(word).vector

In [ ]:
#check
embedding_matrix[0]

In [ ]:
test = nlp('i')

test.vector == embedding_matrix[0]

In [ ]:
embedding_matrix.shape

In [ ]:
#create list of output variables
#e,a,o,c,n

y_train_list = [y_E_train, y_A_train, y_O_train, y_C_train, y_N_train]

y_test_list = [y_E_test, y_A_test, y_O_test, y_C_test, y_N_test]

## Embedding Layer

In [ ]:
#Load the embedding matrix as the weights matrix for the embedding layer and set trainable to False
Embedding_layer= Embedding(input_dim = voc_size, output_dim = embedding_dim, 
weights = [embedding_matrix],
input_length = max_length,
trainable=False)

# Manual LSTM Hyperparameter Search

In [ ]:
model = Sequential()
model.add(Embedding_layer)
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(1, activation='linear'))
model.compile(loss = 'mean_squared_error', optimizer='adam')

print(model.summary())

In [ ]:
from pandas import DataFrame

#training loop
#can handle multiple outcome varaibles and a single model setting

#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights = True)


cor = []

for i in range(0, len(y_train_list)):
  history = model.fit(lstm_x_train, y_train_list[i], epochs=2, batch_size = 32, validation_split = 0.1)
  y_pred = model.predict(lstm_x_test)

  print()
  print('Training Performance')
  print('=======================================')
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()




  print('\n Results Below')
  print('=======================================')

  y_flat = y_pred.flatten()
  r = stats.pearsonr(y_test_list[i], y_flat)

  print('r: ', round(r[0],4))
  cor.append(r)


lstm_results = pd.DataFrame(columns = ['r'])

lstm_results['r'] = cor

from google.colab import files
lstm_results.to_csv('lstm_output_10_15.csv') 
files.download('lstm_output_10_15.csv')


print('Results Downloaded')



In [ ]:
model = Sequential()
model.add(Embedding_layer)
model.add(Bidirectional(LSTM(300)))
model.add(Dropout(0.2))
#model.add(Dense(100, activation = 'relu'))
model.add(Dense(50, activation= 'relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss = 'mean_squared_error', optimizer='adam')

print(model.summary())

In [ ]:
#training loop
#can handle multiple outcome varaibles and a single model setting

#callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights = True)


cor = []

for i in range(0, len(y_train_list)):
  history = model.fit(lstm_x_train, y_train_list[i], epochs=2, batch_size = 32, validation_split = 0.1)
  y_pred = model.predict(lstm_x_test)

  print()
  print('Training Performance')
  print('=======================================')
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['train', 'val'], loc='upper left')
  plt.show()




  print('\n Results Below')
  print('=======================================')

  y_flat = y_pred.flatten()
  r = stats.pearsonr(y_test_list[i], y_flat)

  print('r: ', round(r[0],4))
  cor.append(r)

lstm_results = pd.DataFrame(columns = ['r'])

lstm_results['r'] = cor

from google.colab import files
lstm_results.to_csv('lstm_output_bidirectional.csv') 
files.download('lstm_output_bidirectional.csv')


print('Results Downloaded')



In [ ]:
y_flat.shape

In [ ]:
y_pred.shape

# LSTM Random Grid Search

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier

In [ ]:
def create_model(num_neurons, dropout_rate):
  model = Sequential()
  model.add(Embedding_layer)
  model.add(LSTM(num_neurons))
  model.add(Dropout(dropout_rate))
  #model.add(Dense(100, activation = 'relu'))
  #model.add(Dropout(0.6))
  model.add(Dense(1, activation='linear'))
  model.compile(loss = 'mean_squared_error', optimizer='adam')

  print(model.summary())
  
  return model

In [ ]:
neurons = np.arange(50, 300, 50)
dropout = np.arange(.1, .7, .1)

param_grid = dict(num_neurons = neurons, dropout_rate = dropout)

model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=16, verbose=2)

grid = RandomizedSearchCV(model, param_distributions = param_grid, scoring = 'explained_variance', cv = 5, n_iter = 60, random_state = 152)

In [ ]:
def lstm_train_test(grid, train_x, y_train, x_test, y_test):

  #fit grid
  grid_result = grid.fit(train_x, y_train)

  #track results
  grid_result.best_params_

  best_parameters = grid_result.best_params_
  best_estimator = grid_result.best_estimator_
    
  print('Grid Search Complete')
  print('==================================')
  
  ##### predict on test data
  y_pred = best_estimator.predict(x_test)

  r = pearsonr(y_test, y_pred)
  print('r: ', r)

  return r

In [ ]:
#run optimization across all models

from tqdm import tqdm

n_iterations = 50

results = []

for i in tqdm(range(0, len(y_train_list))):

  for n in tqdm(range(n_iterations)):

    x_boot, y_boot = resample(lstm_x_train, y_train_list[i], replace=True)
    
    acc = lstm_train_test(grid, x_boot, y_boot, lstm_x_test, y_test_list[i])

    print(acc)

    results.append(acc)

# Big Bird

In [ ]:
from transformers import BigBirdTokenizer, \
BigBirdForSequenceClassification, Trainer, TrainingArguments,EvalPrediction, AutoTokenizer

# Dataset Creation

In [ ]:
#prepare train data
e_bert_train_df = train[['Response', 'E_Scale_score']].rename(columns = {'Response' : 'text', 'E_Scale_score': 'labels'})
a_bert_train_df = train[['Response', 'A_Scale_score']].rename(columns = {'Response' : 'text', 'A_Scale_score': 'labels'})
o_bert_train_df = train[['Response', 'O_Scale_score']].rename(columns = {'Response' : 'text', 'O_Scale_score': 'labels'})
c_bert_train_df = train[['Response', 'C_Scale_score']].rename(columns = {'Response' : 'text', 'C_Scale_score': 'labels'})
n_bert_train_df = train[['Response', 'N_Scale_score']].rename(columns = {'Response' : 'text', 'N_Scale_score': 'labels'})

bert_train_list = [e_bert_train_df, a_bert_train_df, o_bert_train_df, c_bert_train_df, n_bert_train_df]

In [ ]:
e_bert_test_df = test[['Response', 'E_Scale_score']].rename(columns = {'Response' : 'text', 'E_Scale_score': 'labels'})
a_bert_test_df = test[['Response', 'A_Scale_score']].rename(columns = {'Response' : 'text', 'A_Scale_score': 'labels'})
o_bert_test_df = test[['Response', 'O_Scale_score']].rename(columns = {'Response' : 'text', 'O_Scale_score': 'labels'})
c_bert_test_df = test[['Response', 'C_Scale_score']].rename(columns = {'Response' : 'text', 'C_Scale_score': 'labels'})
n_bert_test_df = test[['Response', 'N_Scale_score']].rename(columns = {'Response' : 'text', 'N_Scale_score': 'labels'})

bert_test_list = [e_bert_test_df, a_bert_test_df, o_bert_test_df, c_bert_test_df, n_bert_test_df]

In [ ]:
def compute_metrics(eval_preds):

    """
    Compute correlation and mean squared error for regression.
    
    Args:
        eval_preds (EvalPrediction): The evaluation predictions from the Trainer.
        
    Returns:
        dict: Dictionary containing the computed metrics.
    """
    preds, labels = eval_preds.predictions, eval_preds.label_ids
    preds = np.squeeze(preds)  # Remove unnecessary dimensions
    
    # Calculate Pearson correlation coefficient
    correlation = np.corrcoef(preds, labels)[0, 1]
    
    # Return the metrics as a dictionary
    return {
        'correlation': correlation,
    }


def transformer(model, tokenizer, train_dataset, test_dataset, max_length = 4096):

    train_dataset = Dataset.from_pandas(train_dataset)
    test_dataset = Dataset.from_pandas(test_dataset)

    tokenizer = AutoTokenizer.from_pretrained(model)
    model = AutoModelForSequenceClassification.from_pretrained(tokenizer, num_labels=1)

    def tokenize_function(examples, tokenizer):
        return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=max_length)

    train_dataset = train_dataset.map(
      lambda examples: tokenize_function(examples, tokenizer),
      batched=True,
      remove_columns=["text"],
  )

    test_dataset = test_dataset.map(
        lambda examples: tokenize_function(examples, tokenizer),
        batched=True,
        remove_columns=["text"],
  )

    # Define Trainer
    training_args = TrainingArguments(
        output_dir='./results',
        evaluation_strategy="epoch",
        do_eval = False,
        save_total_limit=2,
        learning_rate=2e-5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=64,
        num_train_epochs=2,
        fp16 = True,
        logging_dir='./logs',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        compute_metrics = compute_metrics
    )

    # Train the model
    trainer.train()

    # Evaluate on the test set using the compute_metrics function
    test_results = trainer.evaluate(test_dataset)

    print("Test results:", test_results)

    return test_results

def multi_transformer(train_datasets: list, test_datasets: list, model, tokenizer):
    evaluation_results = []

    for train, test in zip(train_datasets, test_datasets):
        test_results = transformer(model, tokenizer, train, test, max_length = 4096)
        evaluation_results.append(test_results)
    df = pd.DataFrame(evaluation_results)
    df.to_csv('transformer_results.csv', index=False)

# Train Models

In [ ]:
config = {

    'model': "google/bigbird-roberta-base",
    'tokenizer': "google/bigbird-roberta-base"
}

multi_transformer(bert_train_list, bert_test_list, config['model'],  config['model'])